In [1]:
from bs4 import BeautifulSoup
from os import walk

import difflib
import re

In [2]:
pages_list = []

for dirpath, dirnames, filenames in walk('data/raw_pages'):
    if '.ipynb_checkpoints' in dirpath:
        continue
    
    dirpath = dirpath.replace('\\', '/') # для Windows
    for fn in filenames:
        if '.DS_Store' in fn:
            continue 
            
        fp = f'{dirpath}/{fn}'
        pages_list.append(fp)

In [3]:
pages_list[:10]

['data/raw_pages/www.kinopoisk.ru/media/video/4005161.html',
 'data/raw_pages/www.kinopoisk.ru/media/video/4005135.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005182.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005141.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005167.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005148.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005152.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005169.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005165.html',
 'data/raw_pages/www.kinopoisk.ru/media/news/4005162.html']

In [4]:
def remove_script(file):
    soup = BeautifulSoup(''.join(file), 'html.parser')
    for s in soup.select('script'):
        s.extract()
        
    for f in soup.select('footer'):
        f.extract()
    
    for f in soup.select('header'):
        f.extract()
        
    return str(soup).split('\n')

In [5]:
fp_1 = 'data/raw_pages/www.kinopoisk.ru/media/video/4005161.html'
fp_2 = 'data/raw_pages/www.kinopoisk.ru/media/video/4005135.html'

with open(fp_1, 'r') as f:
    file1 = remove_script(f.readlines())
with open(fp_2, 'r') as f:
    file2 = remove_script(f.readlines())

In [6]:
def clean_diff(diff):
    diff = re.sub('\<[^\<\>]+\>', ' ', diff)
    diff = re.sub('&nbsp;', ' ', diff)
    diff = re.sub('\xa0', ' ', diff)
    diff = re.sub('\s\s+', ' ', diff)
    diff = re.sub('^[\+\-] ', '', diff)
    
    return diff

In [7]:
page_lines = []
for diff in difflib.ndiff(file1, file2):
    if re.search('^\+ ', diff) is None:
        continue
        
    diff = clean_diff(diff)
    
    if len(diff) == 0:
        continue
    
    page_lines.append(diff)
#     print(diff)

page_text = ' '.join(page_lines)

print(page_text)

Как снимает Дени Вильнёв и чем хороша «Дюна» — Видео на КиноПоиске  Видео • Как снимает Дени Вильнёв и чем хороша «Дюна» 18 сентября Обсудить 0 Как Дени Вильнёв экспериментирует с жанрами? Что объединяет всех его героев? Почему сдержанный стиль режиссера подходит для исследования тем насилия, жестокости, хрупкости границы между добром и злом? И какой получилась экранизация «Дюны» ? В этом видеоэссе разбираем основные темы и приемы франкоканадского режиссера Дени Вильнёва и пытаемся понять, как ему удается совмещать голливудский размах и авторскую интонацию. Автор: Василий Покровский, Анастасия Усанова Обсудить Пережив встречу с андроидом, полицейские переезжают в Москву. Продолжение народной сай-фай комедии В главных ролях: Зоя Бербер, Антон Филипенко, Сергей Гармаш, Кузьма Сапрыкин, Федор Лавров Режиссер: Максим Пежемский, Александр Карпиловский Смотрите по подписке Смотреть Буду смотреть Смотрите также Фильм недели: черви и звезды в «Дюне» — главном блокбастере года 3 сентября 102 Гд

In [8]:
def extract_page_text(file, origin_file):
    
    page_lines = []
    for diff in difflib.ndiff(origin_file, file):
        if re.search('^\+ ', diff) is None:
            continue

        diff = clean_diff(diff)

        if len(diff) == 0:
            continue

        page_lines.append(diff)

    page_text = '. '.join(page_lines)
    
    return page_text

In [10]:
# Извлекаем тексты, и сохраняем в файлы

page_texts_path = 'data/page_texts/'
create_fp = lambda fp: fp.replace('/', '+').replace('\\', '+') + '.txt'


with open(pages_list[0], 'r', encoding="utf-8") as f:
    origin_file = remove_script(f.readlines())

with open(pages_list[1], 'r', encoding="utf-8") as f:
    file = remove_script(f.readlines())

page_text = extract_page_text(origin_file, file)

with open(page_texts_path + create_fp(pages_list[0]), 'w', encoding="utf-8") as f:
    f.write(page_text)

for fp in pages_list[1:]:
    with open(fp, 'r', encoding="utf-8") as f:
        file = remove_script(f.readlines())
    
    page_text = extract_page_text(file, origin_file)
    with open(page_texts_path + create_fp(fp), 'w', encoding="utf-8") as f:
        f.write(page_text)

In [13]:
pages_texts = {}

for dirpath, dirnames, filenames in walk(page_texts_path):
    if '.ipynb_checkpoints' in dirpath:
        continue
        
    for fn in filenames:
        if '.DS_Store' in fn:
            continue
            
        fp = f'{dirpath}/{fn}'
        with open(fp, 'r', encoding="utf-8") as f:
            pt = ' '.join(f.readlines())
        
        original_link = fp.split('/')[-1].replace('|', '/').replace('.txt', '')
        pages_texts[original_link] = pt

In [96]:
!pip install pymorphy2
!pip install pymorphy2-dicts

# ставим либо DAWG-Python, либо DAWG (он быстрее)
!pip install DAWG-Python

# Если есть Cython и компилятор в системе
# !pip install DAWG

     |████████████████████████████████| 55 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 8.2 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.1 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 176 kB 1.6 MB/s eta 0:00:01


In [21]:
import nltk
import pymorphy2
from string import punctuation

print(punctuation)

MORPH = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub('\s\s+', ' ', text)
    
    return text

def extract_sentences(text):
    text = preprocess_text(text)

    sentences = []
    for s in nltk.sent_tokenize(text):
        wrds = []
        for wrd in nltk.word_tokenize(s):
            if wrd in punctuation:
                continue
            
            wrd = MORPH.parse(wrd)[0].normal_form
            wrds.append(wrd)
            
        clear_sentence = ' '.join(wrds)
        
        sentences.append(clear_sentence)
    
    return sentences

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [25]:
MORPH.parse('мамой')[0]

Parse(word='мамой', tag=OpencorporaTag('NOUN,anim,femn sing,ablt'), normal_form='мама', score=1.0, methods_stack=((DictionaryAnalyzer(), 'мамой', 1988, 4),))

In [26]:
nltk.sent_tokenize('я люблю тебя до слёз. будь счастлива!')

['я люблю тебя до слёз.', 'будь счастлива!']

In [23]:
nltk.word_tokenize('мама, мыла раму папу-макаку за 170,00 рублей 170рублей')

['мама',
 ',',
 'мыла',
 'раму',
 'папу-макаку',
 'за',
 '170,00',
 'рублей',
 '170рублей']

In [29]:
import pandas as pd
import numpy as np

In [27]:
len(pages_texts)

44

In [37]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arthursaprykin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
full_link_df = pd.DataFrame(columns=['link', 'sentence_order', 'sentence_text'])

for link, text in pages_texts.items():
    sentences = extract_sentences(text)
    sentence_count = len(sentences)
    
    link_df = pd.DataFrame({'link': np.repeat(link, sentence_count),
                            'sentence_order': np.arange(sentence_count),
                            'sentence_text': sentences})
    
    full_link_df = full_link_df.append(link_df)

In [41]:
full_link_df

,link,sentence_order,sentence_text
0,data/raw_pages/www.kinopoisk.ru/media/article/...,0,6 главный аниме-режиссёр xxi век — статья на к...
1,data/raw_pages/www.kinopoisk.ru/media/article/...,1,6 главный аниме-режиссёр xxi век вчера обсудит...
2,data/raw_pages/www.kinopoisk.ru/media/article/...,2,масаак юас сатося кон исао такахат мамор хосод...
3,data/raw_pages/www.kinopoisk.ru/media/article/...,3,буйный визуальность он работа быть слишком рад...
4,data/raw_pages/www.kinopoisk.ru/media/article/...,4,зато « игра разум » заметить масао маруям соуч...
...,...,...,...
37,data/raw_pages/www.kinopoisk.ru/media/news/400...,37,21 сентябрь 247 интервью дени вильнёв « тимоти...
38,data/raw_pages/www.kinopoisk.ru/media/news/400...,38,почему это важно
39,data/raw_pages/www.kinopoisk.ru/media/news/400...,39,вчера 11 сериал « игра в кальмар » возглавить ...
40,data/raw_pages/www.kinopoisk.ru/media/news/400...,40,чем южнокорейский шоу о выживание понравиться ...


In [35]:
full_link_df.to_csv('data/link_page_sentences.csv', index=False)

In [38]:
pd.read_csv('data/link_page_sentences.csv').head(5)

,link,sentence_order,sentence_text
0,data/raw_pages/www.kinopoisk.ru/media/article/...,0,6 главный аниме-режиссёр xxi век — статья на к...
1,data/raw_pages/www.kinopoisk.ru/media/article/...,1,6 главный аниме-режиссёр xxi век вчера обсудит...
2,data/raw_pages/www.kinopoisk.ru/media/article/...,2,масаак юас сатося кон исао такахат мамор хосод...
3,data/raw_pages/www.kinopoisk.ru/media/article/...,3,буйный визуальность он работа быть слишком рад...
4,data/raw_pages/www.kinopoisk.ru/media/article/...,4,зато « игра разум » заметить масао маруям соуч...


In [42]:
!pip install pyarrow

In [43]:
full_link_df.reset_index().to_feather('data/link_page_sentences.ftr')

In [33]:
pd.read_feather('data/link_page_sentences.ftr').head(5)

,index,link,sentence_order,sentence_text
0,0,data/raw_pages/www.kinopoisk.ru/media/article/...,0,6 главный аниме-режиссёр xxi век — статья на к...
1,1,data/raw_pages/www.kinopoisk.ru/media/article/...,1,6 главный аниме-режиссёр xxi век вчера обсудит...
2,2,data/raw_pages/www.kinopoisk.ru/media/article/...,2,масаак юас сатося кон исао такахат мамор хосод...
3,3,data/raw_pages/www.kinopoisk.ru/media/article/...,3,буйный визуальность он работа быть слишком рад...
4,4,data/raw_pages/www.kinopoisk.ru/media/article/...,4,зато « игра разум » заметить масао маруям соуч...


In [32]:
full_link_df.to_parquet('data/link_page_sentences.parquet')

In [135]:
pd.read_parquet('data/link_page_sentences.parquet').head(5)

,link,sentence_order,sentence_text
0,data/raw_pages/www.kinopoisk.ru/media/article/...,0,6 главный аниме-режиссёр xxi век — статья на к...
1,data/raw_pages/www.kinopoisk.ru/media/article/...,1,6 главный аниме-режиссёр xxi век сегодня обсуд...
2,data/raw_pages/www.kinopoisk.ru/media/article/...,2,масаак юас сатося кон исао такахат мамор хосод...
3,data/raw_pages/www.kinopoisk.ru/media/article/...,3,буйный визуальность он работа быть слишком рад...
4,data/raw_pages/www.kinopoisk.ru/media/article/...,4,зато « игра разум » заметить масао маруям соуч...
